In [1]:
# Import our dependencies
import numpy as np
import pandas as pd
import polars as pl

In [2]:
#  Import and read the input csv
PharmDp10k_df = pd.read_csv('../../Raw_Data/WHO_Pharmacists_per10k-data.csv')
PharmDp10k_df.head(5)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,HWF_0014,"Pharmacists (per 10,000)",numeric,AFR,Africa,Country,GNB,Guinea-Bissau,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.01,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
1,HWF_0014,"Pharmacists (per 10,000)",numeric,AFR,Africa,Country,NER,Niger,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.02,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
2,HWF_0014,"Pharmacists (per 10,000)",numeric,AFR,Africa,Country,CAF,Central African Republic,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.03,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
3,HWF_0014,"Pharmacists (per 10,000)",numeric,AFR,Africa,Country,GMB,Gambia,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.04,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z
4,HWF_0014,"Pharmacists (per 10,000)",numeric,AFR,Africa,Country,TCD,Chad,Year,2023,...,NaN,NaN,NaN,NaN,NaN,0.08,NaN,"NHWA data portal, December 2024 update https:/...",EN,2025-01-14T05:00:00.000Z


In [3]:
# Drop the population estimate column
PharmDp10k_df = PharmDp10k_df.drop(['IndicatorCode','ValueType','ParentLocationCode','ParentLocation','Location type','SpatialDimValueCode','Period type'], axis=1)
PharmDp10k_df = PharmDp10k_df.drop(['FactValueUoM','FactValueNumericLowPrefix','FactValueNumericLow','FactValueNumericHighPrefix','FactValueNumericHigh'], axis=1)
PharmDp10k_df = PharmDp10k_df.drop(['FactValueTranslationID','FactComments','Language','DateModified','IsLatestYear','Dim1 type','Dim1','Dim2','Dim3',], axis=1)
PharmDp10k_df = PharmDp10k_df.drop(['Dim1ValueCode','Dim2ValueCode','Dim3ValueCode','Dim2 type','Dim3 type','DataSource','FactValueNumericPrefix'], axis=1)
PharmDp10k_df = PharmDp10k_df.drop(['FactValueNumeric','DataSourceDimValueCode','Indicator'], axis=1)
PharmDp10k_df.rename(columns ={'Location':'Country', 'Period':'Year', 'Value':'PharmD per 10k'}, inplace = True)
PharmDp10k_df

,Country,Year,PharmD per 10k
0,Guinea-Bissau,2023,0.01
1,Niger,2023,0.02
2,Central African Republic,2023,0.03
3,Gambia,2023,0.04
4,Chad,2023,0.08
...,...,...,...
2871,Kazakhstan,1990,8.50
2872,Iceland,1990,8.75
2873,Spain,1990,9.36
2874,France,1990,9.61


In [4]:
# Open the list of country name corrections
corrections_df = pd.read_csv('../../Clean_Data/master_country_list/country_name_corrections.csv')

# Convert the corrections dataframe to a dictionary.
correction_dict = dict(zip(corrections_df['wrong'], corrections_df['correct']))

In [5]:
# Apply the correction dictionary to fix the known errors
PharmDp10k_df['Country'] = PharmDp10k_df['Country'].replace(correction_dict)

In [6]:
# Open the master list of countries
countries_df = pd.read_csv('../../Clean_Data/master_country_list/country_profile_urls.csv')
countries_df = countries_df.drop(['profile_url'], axis=1)

In [7]:
country_list = countries_df['country'].tolist()
#country_list

In [8]:
# Create a list of MDp10k countries
PharmDp10k_countries = PharmDp10k_df['Country'].tolist()
#NMp10k_countries

In [9]:
# clean the FS data based on the SS country list
no_match = []

for country in PharmDp10k_countries:
    if country in country_list:
        continue
    else:
        no_match.append(country)

In [10]:
PharmDp10k_clean_df = PharmDp10k_df.drop(PharmDp10k_df[PharmDp10k_df['Country'].isin(no_match)].index.tolist())
PharmDp10k_clean_df

,Country,Year,PharmD per 10k
0,Guinea-Bissau,2023,0.01
1,Niger,2023,0.02
2,Central African Republic,2023,0.03
3,Gambia,2023,0.04
4,Chad,2023,0.08
...,...,...,...
2871,Kazakhstan,1990,8.50
2872,Iceland,1990,8.75
2873,Spain,1990,9.36
2874,France,1990,9.61


In [11]:
PharmDp10k_sorted_df = PharmDp10k_clean_df.sort_values(['Country','Year'], axis=0)

In [12]:
PharmDp10k_sorted_df.tail(40)

,Country,Year,PharmD per 10k
1840,Vietnam,2006,1.980
1760,Vietnam,2007,2.160
1662,Vietnam,2008,2.320
1550,Vietnam,2009,2.500
1442,Vietnam,2010,2.690
1247,Vietnam,2012,4.550
1039,Vietnam,2014,3.400
938,Vietnam,2015,3.400
829,Vietnam,2016,3.370
2505,Yemen,1997,0.330


In [13]:
# Create a complete dataframe with all countries and all the years.

# Define variables to collect the details.
#data_countries = MDp10k_sorted_df['Country'].unique()
data_countries = country_list
years = list(range(1990, 2024))

# Create a helper DataFrame.
helper_df = pd.DataFrame({'Country':np.repeat(data_countries, len(years)), 'Year':np.tile(years, len(data_countries))})

# Merge the helper DataFrame with the original data to ensure the data range is complete.
complete_df = pd.merge(helper_df, PharmDp10k_sorted_df, on=['Country','Year'], how='left')

In [14]:
# Convert to wide format
wide_df = complete_df.pivot(index='Year', columns='Country', values='PharmD per 10k')

# Flatten the hierarchical columns and create 'Country_Series' style column names
wide_df.columns = [f'{col}_PharmD per 10k' for col in wide_df.columns]

# Reset the index, so 'Year' becomes a column again
wide_df.reset_index(inplace=True)

# Convert columns to numeric, coerce non-numeric values to NaN
for col in wide_df.columns[0:]:
    wide_df[col] = pd.to_numeric(wide_df[col], errors='coerce')

# Convert to Polars DataFrame
wide_pl_df = pl.from_pandas(wide_df)
    
wide_pl_df.head(5)

Year,Albania_PharmD per 10k,Algeria_PharmD per 10k,Andorra_PharmD per 10k,Angola_PharmD per 10k,Antigua and Barbuda_PharmD per 10k,Argentina_PharmD per 10k,Armenia_PharmD per 10k,Aruba_PharmD per 10k,Australia_PharmD per 10k,Austria_PharmD per 10k,Azerbaijan_PharmD per 10k,Bahamas_PharmD per 10k,Bahrain_PharmD per 10k,Bangladesh_PharmD per 10k,Barbados_PharmD per 10k,Belarus_PharmD per 10k,Belgium_PharmD per 10k,Belize_PharmD per 10k,Benin_PharmD per 10k,Bermuda_PharmD per 10k,Bhutan_PharmD per 10k,Bolivia_PharmD per 10k,Bosnia and Herzegovina_PharmD per 10k,Botswana_PharmD per 10k,Brazil_PharmD per 10k,British Virgin Islands_PharmD per 10k,Brunei_PharmD per 10k,Bulgaria_PharmD per 10k,Burkina Faso_PharmD per 10k,Burundi_PharmD per 10k,Cabo Verde_PharmD per 10k,Cambodia_PharmD per 10k,Cameroon_PharmD per 10k,Canada_PharmD per 10k,Central African Republic_PharmD per 10k,Chad_PharmD per 10k,…,Serbia_PharmD per 10k,Seychelles_PharmD per 10k,Sierra Leone_PharmD per 10k,Singapore_PharmD per 10k,Slovakia_PharmD per 10k,Slovenia_PharmD per 10k,Solomon Islands_PharmD per 10k,South Africa_PharmD per 10k,South Korea_PharmD per 10k,Spain_PharmD per 10k,Sri Lanka_PharmD per 10k,Sudan_PharmD per 10k,Suriname_PharmD per 10k,Sweden_PharmD per 10k,Switzerland_PharmD per 10k,Syria_PharmD per 10k,Taiwan (China)_PharmD per 10k,Tajikistan_PharmD per 10k,Tanzania_PharmD per 10k,Thailand_PharmD per 10k,Togo_PharmD per 10k,Trinidad and Tobago_PharmD per 10k,Tunisia_PharmD per 10k,Turkmenistan_PharmD per 10k,Türkiye_PharmD per 10k,Uganda_PharmD per 10k,Ukraine_PharmD per 10k,United Kingdom_PharmD per 10k,United States_PharmD per 10k,Uruguay_PharmD per 10k,Uzbekistan_PharmD per 10k,Vanuatu_PharmD per 10k,Venezuela_PharmD per 10k,Vietnam_PharmD per 10k,Yemen_PharmD per 10k,Zambia_PharmD per 10k,Zimbabwe_PharmD per 10k
i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990,3.65,null,null,null,null,null,2.12,null,5.26,4.54,6.25,null,null,null,null,3.2,8.2,null,null,null,null,null,1.83,null,null,null,null,4.95,null,null,null,null,null,6.8,null,null,…,null,null,null,null,null,3.26,null,null,null,9.36,null,null,null,5.94,5.31,null,null,1.16,null,null,null,null,1.49,null,2.82,null,null,null,null,null,null,null,null,null,null,null,0.34
1991,3.75,null,null,null,null,null,2.16,null,6.8,4.62,6.09,null,null,null,null,3.38,8.43,null,null,null,null,null,1.82,null,null,null,null,3.72,null,null,null,null,null,6.63,null,null,…,null,null,null,null,null,null,null,null,null,9.6,null,null,null,6.13,5.44,null,null,1.25,null,0.78,null,null,1.53,null,2.81,null,null,null,null,null,null,null,null,null,null,null,null
1992,3.81,null,null,null,null,3.12,2.05,null,5.76,4.78,6.18,null,null,null,null,3.47,8.6,null,null,null,null,null,null,null,null,null,null,3.12,null,null,null,null,null,6.8,null,null,…,null,null,null,null,null,null,null,null,null,9.82,null,null,null,6.25,5.55,null,null,1.13,null,null,null,null,1.53,null,2.86,null,null,null,null,null,null,null,null,null,null,null,null
1993,3.89,null,null,null,null,null,1.65,null,5.75,4.9,5.93,null,null,null,null,3.46,8.84,null,null,null,null,null,null,null,null,null,null,2.81,null,null,null,null,null,6.91,null,null,…,null,null,null,null,null,3.59,null,null,null,9.99,0.33,null,null,6.43,5.73,null,null,0.4,null,0.82,null,null,1.49,null,3.0,null,null,null,null,null,null,null,null,null,null,null,null
1994,3.98,null,null,null,null,null,1.46,null,5.94,5.05,5.9,null,null,null,null,3.47,9.08,null,null,null,null,null,null,null,null,null,null,2.47,null,null,null,null,null,6.97,null,null,…,null,null,null,null,null,4.92,null,null,null,10.12,null,null,null,6.59,5.91,null,null,0.34,null,null,null,null,1.57,null,3.06,null,null,null,null,null,null,null,null,null,null,null,null


In [15]:
# Handle the missing data

wide_pl_df = wide_pl_df.interpolate()
wide_pl_df = wide_pl_df.fill_null(strategy='backward')
wide_pl_df = wide_pl_df.fill_null(strategy='forward')
wide_pl_df = wide_pl_df.fill_null(strategy='zero')

In [16]:
wide_pl_df.head(15)

Year,Albania_PharmD per 10k,Algeria_PharmD per 10k,Andorra_PharmD per 10k,Angola_PharmD per 10k,Antigua and Barbuda_PharmD per 10k,Argentina_PharmD per 10k,Armenia_PharmD per 10k,Aruba_PharmD per 10k,Australia_PharmD per 10k,Austria_PharmD per 10k,Azerbaijan_PharmD per 10k,Bahamas_PharmD per 10k,Bahrain_PharmD per 10k,Bangladesh_PharmD per 10k,Barbados_PharmD per 10k,Belarus_PharmD per 10k,Belgium_PharmD per 10k,Belize_PharmD per 10k,Benin_PharmD per 10k,Bermuda_PharmD per 10k,Bhutan_PharmD per 10k,Bolivia_PharmD per 10k,Bosnia and Herzegovina_PharmD per 10k,Botswana_PharmD per 10k,Brazil_PharmD per 10k,British Virgin Islands_PharmD per 10k,Brunei_PharmD per 10k,Bulgaria_PharmD per 10k,Burkina Faso_PharmD per 10k,Burundi_PharmD per 10k,Cabo Verde_PharmD per 10k,Cambodia_PharmD per 10k,Cameroon_PharmD per 10k,Canada_PharmD per 10k,Central African Republic_PharmD per 10k,Chad_PharmD per 10k,…,Serbia_PharmD per 10k,Seychelles_PharmD per 10k,Sierra Leone_PharmD per 10k,Singapore_PharmD per 10k,Slovakia_PharmD per 10k,Slovenia_PharmD per 10k,Solomon Islands_PharmD per 10k,South Africa_PharmD per 10k,South Korea_PharmD per 10k,Spain_PharmD per 10k,Sri Lanka_PharmD per 10k,Sudan_PharmD per 10k,Suriname_PharmD per 10k,Sweden_PharmD per 10k,Switzerland_PharmD per 10k,Syria_PharmD per 10k,Taiwan (China)_PharmD per 10k,Tajikistan_PharmD per 10k,Tanzania_PharmD per 10k,Thailand_PharmD per 10k,Togo_PharmD per 10k,Trinidad and Tobago_PharmD per 10k,Tunisia_PharmD per 10k,Turkmenistan_PharmD per 10k,Türkiye_PharmD per 10k,Uganda_PharmD per 10k,Ukraine_PharmD per 10k,United Kingdom_PharmD per 10k,United States_PharmD per 10k,Uruguay_PharmD per 10k,Uzbekistan_PharmD per 10k,Vanuatu_PharmD per 10k,Venezuela_PharmD per 10k,Vietnam_PharmD per 10k,Yemen_PharmD per 10k,Zambia_PharmD per 10k,Zimbabwe_PharmD per 10k
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1990.0,3.65,1.71,8.29,0.02,1.84,3.12,2.12,0.0,5.26,4.54,6.25,3.85,1.9,0.55,6.45,3.2,8.2,3.58,0.01,0.0,0.11,2.96,1.83,0.78,3.37,0.0,0.88,4.95,0.17,0.1,0.06,0.3,0.02,6.8,0.04,0.03,…,2.47,0.57,0.16,2.63,3.66,3.26,0.65,2.27,10.83,9.36,0.33,0.44,0.57,5.94,5.31,4.7,0.0,1.16,0.03,0.78,0.09,4.12,1.49,3.21,2.82,0.08,0.29,5.87,8.12,0.65,0.31,0.09,0.0,0.77,0.33,0.23,0.34
1991.0,3.75,1.71,8.29,0.02,1.84,3.12,2.16,0.0,6.8,4.62,6.09,3.85,1.9,0.55,6.45,3.38,8.43,3.58,0.01,0.0,0.11,2.96,1.82,0.78,3.37,0.0,0.88,3.72,0.17,0.1,0.06,0.3,0.02,6.63,0.04,0.03,…,2.47,0.57,0.16,2.63,3.66,3.37,0.65,2.27,10.83,9.6,0.33,0.44,0.57,6.13,5.44,4.7,0.0,1.25,0.03,0.78,0.09,4.12,1.53,3.21,2.81,0.08,0.29,5.87,8.12,0.65,0.31,0.09,0.0,0.77,0.33,0.23,0.364
1992.0,3.81,1.71,8.29,0.02,1.84,3.12,2.05,0.0,5.76,4.78,6.18,3.85,1.9,0.55,6.45,3.47,8.6,3.58,0.01,0.0,0.11,2.96,1.6975,0.78,3.37,0.0,0.88,3.12,0.17,0.1,0.06,0.3,0.02,6.8,0.04,0.03,…,2.47,0.57,0.16,2.63,3.66,3.48,0.65,2.27,10.83,9.82,0.33,0.44,0.57,6.25,5.55,4.7,0.0,1.13,0.03,0.8,0.09,4.12,1.53,3.21,2.86,0.08,0.29,5.87,8.12,0.65,0.31,0.09,0.0,0.77,0.33,0.23,0.388
1993.0,3.89,1.71,8.29,0.02,1.84,3.301667,1.65,0.0,5.75,4.9,5.93,3.85,1.9,0.55,6.45,3.46,8.84,3.58,0.01,0.0,0.11,2.96,1.575,0.78,3.37,0.0,0.88,2.81,0.17,0.1,0.06,0.3,0.02,6.91,0.04,0.03,…,2.47,0.57,0.16,2.63,3.66,3.59,0.65,2.27,10.83,9.99,0.33,0.44,0.57,6.43,5.73,4.7,0.0,0.4,0.03,0.82,0.09,4.12,1.49,3.21,3.0,0.08,0.29,5.87,8.12,0.65,0.31,0.09,0.0,0.77,0.33,0.23,0.412
1994.0,3.98,1.71,8.29,0.02,1.84,3.483333,1.46,0.0,5.94,5.05,5.9,3.85,1.9,0.55,6.45,3.47,9.08,3.58,0.01,0.0,0.11,2.96,1.4525,0.78,3.37,0.0,0.88,2.47,0.17,0.1,0.06,0.3,0.02,6.97,0.04,0.03,…,2.47,0.57,0.16,2.63,3.66,4.92,0.65,2.27,10.83,10.12,0.340909,0.44,0.57,6.59,5.91,4.7,0.0,0.34,0.03,0.905,0.09,4.12,1.57,3.21,3.06,0.08,0.29,5.87,8.12,0.65,0.31,0.09,0.0,0.77,0.33,0.23,0.436
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…

In [18]:
# Reformat the dataframe with the filled values...
# Convert the DataFrame back to pandas!!!
wide_df = wide_pl_df.to_pandas() 

# Now we can reshape it with pandas functionality
complete_df = wide_df.melt(id_vars='Year', value_vars=wide_df.columns[1:], var_name='Country', value_name='PharmD per 10k')

# Removing "_MD per 10k" from country names
complete_df['Country'] = complete_df['Country'].str.replace('_PharmD per 10k', '')

complete_df['Year'] = complete_df['Year'].astype(int)

# Creating 'Country_Year' column
complete_df['Country_Year'] = complete_df['Country'].astype(str) + '_' + complete_df['Year'].astype(str)

complete_df = complete_df.loc[:, ['Country_Year', 'Country', 'Year', 'PharmD per 10k']]
#complete_df['Country_Year'] = complete_df['Country_Year'].astype(str)


In [19]:
complete_df

,Country_Year,Country,Year,PharmD per 10k
0,Albania_1990,Albania,1990,3.6500
1,Albania_1991,Albania,1991,3.7500
2,Albania_1992,Albania,1992,3.8100
3,Albania_1993,Albania,1993,3.8900
4,Albania_1994,Albania,1994,3.9800
...,...,...,...,...
6489,Zimbabwe_2019,Zimbabwe,2019,0.9755
6490,Zimbabwe_2020,Zimbabwe,2020,1.0210
6491,Zimbabwe_2021,Zimbabwe,2021,1.1005
6492,Zimbabwe_2022,Zimbabwe,2022,1.1800


In [20]:
complete_df.to_csv('../../Clean_Data/Clean_CSV_Files/PharmD_per_10k.csv', index=False, encoding="utf-8")